In [1]:
# Import libraries.
import pandas as pd

In [2]:
# Read input data.
cust_data = pd.read_csv("raw_data/customer_data.csv")
trans_data = pd.read_csv("raw_data/transaction_data.csv")

# FX rate are for 1 GBP and assumes no change over time for simplicity.
fx_data = pd.read_csv("raw_data/fx_gbp.csv",header=1, skiprows=[0,2], usecols=["currency", "rate"])

# len(fx_data.currency.unique())
# len(trans_data.currency.unique())

In [3]:
print(fx_data.head())

  currency         rate
0      THB    37.386304
1      BIF  2242.924895
2      NGN   439.534518
3      LKR   214.801634
4      CHF     1.194301


In [4]:
print(trans_data.head())

   purchase_id  buyer_id country  product_id   department     category  \
0            3  10258663      AU   113961581  Accessories  Accessories   
1            6   9135670      AU   114780505  Accessories  Accessories   
2            9  10869093      AU   115216176  Accessories  Accessories   
3           27  10182131      AU   112217251  Accessories  Accessories   
4           31   5766224      AU   107519551  Accessories  Accessories   

  purchase_date currency  value_of_item  
0    02/08/2019      AUD           15.0  
1    02/08/2019      AUD           33.5  
2    07/08/2019      AUD           65.0  
3    02/08/2019      AUD           40.0  
4    02/08/2019      AUD           25.0  


In [5]:
print(cust_data.head())

   buyer_id gender  membership_length age_group
0       505      F                0.2     50-60
1      1989      F                0.8       60+
2      2234      F                0.4     50-60
3      2619      F                0.4     50-60
4      3392      F                2.3     50-60


In [6]:
# Join trans_data with fx_gbp for fixed currency for analysis.
abt_trans = pd.merge(left=trans_data, right=fx_data, how="left", on="currency")
print(abt_trans.head())

   purchase_id  buyer_id country  product_id   department     category  \
0            3  10258663      AU   113961581  Accessories  Accessories   
1            6   9135670      AU   114780505  Accessories  Accessories   
2            9  10869093      AU   115216176  Accessories  Accessories   
3           27  10182131      AU   112217251  Accessories  Accessories   
4           31   5766224      AU   107519551  Accessories  Accessories   

  purchase_date currency  value_of_item      rate  
0    02/08/2019      AUD           15.0  1.799521  
1    02/08/2019      AUD           33.5  1.799521  
2    07/08/2019      AUD           65.0  1.799521  
3    02/08/2019      AUD           40.0  1.799521  
4    02/08/2019      AUD           25.0  1.799521  


In [7]:
# Check if there are any missing rates from the join.
sum(abt_trans['rate'].isna()) # Output shows all good.

0

In [8]:
# EDA
# Check if currency = 'GBP' use only in country = 'GB'.
print(abt_trans[(abt_trans['currency'] == 'GBP') & (abt_trans['country'] != 'GB')].head())

# Consumers can pay in different currencies to.

      purchase_id  buyer_id country  product_id   department     category  \
1351        15394  15244480     RoW    47083009  Accessories  Accessories   
1386        15844  10237835      DE   115404303  Accessories  Accessories   
1418        16215  12271165     RoW   114976510  Accessories  Accessories   
1424        16292   9758568      DE   104409015  Accessories  Accessories   
1425        16324   6088950     RoW   114404956  Accessories  Accessories   

     purchase_date currency  value_of_item  rate  
1351    07/08/2019      GBP          51.00   1.0  
1386    07/08/2019      GBP           8.00   1.0  
1418    07/08/2019      GBP          90.00   1.0  
1424    02/08/2019      GBP         158.99   1.0  
1425    01/08/2019      GBP          34.00   1.0  


In [9]:
# EDA
# Calculate count of payments from each unique country current. Sorts by country ascending and count descending.

payment_types = abt_trans[['country','currency']]
unique_payment_types = payment_types.groupby(list(payment_types.columns)).size().reset_index(name='count').sort_values(by=['country', 'count'], ascending=[True, False])

unique_payment_types

,country,currency,count
0,AU,AUD,6828
4,AU,GBP,270
7,AU,USD,182
3,AU,EUR,18
1,AU,CAD,14
...,...,...,...
60,US,HUF,1
62,US,MYR,1
64,US,SEK,1
66,US,THB,1


In [10]:
abt_trans['gbp_value_of_item'] = abt_trans.value_of_item * abt_trans.rate
abt_trans.head()

,purchase_id,buyer_id,country,product_id,department,category,purchase_date,currency,value_of_item,rate,gbp_value_of_item
0,3,10258663,AU,113961581,Accessories,Accessories,02/08/2019,AUD,15.0,1.799521,26.992815
1,6,9135670,AU,114780505,Accessories,Accessories,02/08/2019,AUD,33.5,1.799521,60.283953
2,9,10869093,AU,115216176,Accessories,Accessories,07/08/2019,AUD,65.0,1.799521,116.968865
3,27,10182131,AU,112217251,Accessories,Accessories,02/08/2019,AUD,40.0,1.799521,71.980840
4,31,5766224,AU,107519551,Accessories,Accessories,02/08/2019,AUD,25.0,1.799521,44.988025


In [11]:
# cust_data.shape #(129113,4)
# len(trans_data['buyer_id'].unique()) #129283

In [12]:
abt = abt_trans = pd.merge(left=abt_trans, right=cust_data, how="left", on="buyer_id")
# abt

In [13]:
abt.to_csv('cleaned_data/abt.csv', encoding='utf-8', index=False)
abt_trans.to_csv('cleaned_data/abt_trans.csv', encoding='utf-8', index=False)

In [13]:
abt.purchase_date.unique()

# 01/08/2019 = Thursday
# 02/08/2019 = Friday
# 03/08/2019 = Saturday
# 04/08/2019 = Sunday
# 05/08/2019 = Monday
# 06/08/2019 = Tuesday
# 07/08/2019 = Wednesday
# 03/08/2019 = Saturday

array(['02/08/2019', '07/08/2019', '03/08/2019', '01/08/2019',
       '06/08/2019', '04/08/2019', '05/08/2019'], dtype=object)

In [14]:
abt.department.unique()


array(['Accessories', 'Beauty', 'Mens apparel', 'Womens apparel', 'Home',
       'Jewellery', 'Kids', 'Shoes', 'Tech'], dtype=object)

In [25]:
cat_dep = abt[['category', 'department']].drop_duplicates().sort_values(by=['department', 'category'])
#.size().reset_index(name='count').sort_values(by=['category', 'department'], ascending=[True, False])
print(cat_dep)

                category      department
0            Accessories     Accessories
20690             Beauty          Beauty
81942               Home            Home
85336          Jewellery       Jewellery
99893               Kids            Kids
28926     Bottoms - Mens    Mens apparel
113246  Outerwear - Mens    Mens apparel
141930       Tops - Mens    Mens apparel
117320             Shoes           Shoes
138654              Tech            Tech
34629   Bottoms - Womens  Womens apparel
63681            Dresses  Womens apparel
101989          Lingerie  Womens apparel
107135         Outerwear  Womens apparel
174012     Tops - Womens  Womens apparel


In [ ]:
abt.head(), abt.info(), abt.describe()

In [ ]:
# Demographic Breakdown: Gender, Age Group, Membership Length

# Grouping by gender, age group, and membership length to analyze purchase patterns
gender_analysis = data.groupby('gender')['gbp_value_of_item'].agg(['count', 'sum', 'mean'])
age_group_analysis = data.groupby('age_group')['gbp_value_of_item'].agg(['count', 'sum', 'mean'])
membership_length_analysis = data.groupby(pd.cut(data['membership_length'], bins=[0, 1, 5, 10, 15]))['gbp_value_of_item'].agg(['count', 'sum', 'mean'])

# Display the results
gender_analysis, age_group_analysis, membership_length_analysis


In [ ]:
# Analyze the demographic breakdown based on gender and age group
gender_distribution = data['gender'].value_counts()
age_group_distribution = data['age_group'].value_counts()

# Plot the distribution of purchases by gender and age group
fig, axs = plt.subplots(1, 2, figsize=(14, 6))

# Gender distribution
axs[0].pie(gender_distribution, labels=gender_distribution.index, autopct='%1.1f%%', colors=['lightcoral', 'lightskyblue'])
axs[0].set_title('Purchases by Gender')

# Age group distribution
axs[1].bar(age_group_distribution.index, age_group_distribution.values, color='mediumseagreen')
axs[1].set_title('Purchases by Age Group')
axs[1].set_xlabel('Age Group')
axs[1].set_ylabel('Number of Purchases')

plt.tight_layout()
plt.show()

# Analyze membership length distribution
membership_summary = data['membership_length'].describe()

gender_distribution, age_group_distribution, membership_summary

In [ ]:
# Product Categories & Departments Analysis

# Grouping by department and category to analyze purchase trends
department_analysis = data.groupby('department')['gbp_value_of_item'].agg(['count', 'sum', 'mean']).sort_values(by='sum', ascending=False)
category_analysis = data.groupby('category')['gbp_value_of_item'].agg(['count', 'sum', 'mean']).sort_values(by='sum', ascending=False)

# Display the results
department_analysis, category_analysis


In [ ]:
# Purchase Patterns: High-Value Customers and Common Purchase Behaviors

# Identifying high-value customers based on total spending
customer_spending = data.groupby('buyer_id')['gbp_value_of_item'].agg(['count', 'sum', 'mean']).sort_values(by='sum', ascending=False)

# Calculating overall purchase frequency and average spending
overall_purchase_behavior = {
    'total_purchases': data['purchase_id'].count(),
    'total_customers': data['buyer_id'].nunique(),
    'average_purchases_per_customer': data.groupby('buyer_id')['purchase_id'].count().mean(),
    'average_spend_per_customer': customer_spending['sum'].mean(),
    'average_spend_per_purchase': data['gbp_value_of_item'].mean()
}

# Displaying the top 10 high-value customers and the overall purchase behavior
high_value_customers = customer_spending.head(10)
high_value_customers, overall_purchase_behavior
